<a href="https://colab.research.google.com/drive/1RvrnYg5Tpu81hNSnjuLdeF5VWxC_NWRv?usp=sharing">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open in Colab"/>
</a>

In [1]:

#1 Install libraries
!pip install -q unsloth "torch>=2.1" transformers peft datasets trl accelerate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.4/46.4 kB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.2/193.2 kB 11.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 63.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 60.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 58.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 74.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
#2 Load TinyLlama with Unsloth

from unsloth import FastLanguageModel
import torch

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name="TinyLlama/TinyLlama-1.1B-Chat-v1.0",
    max_seq_length=2048,
    dtype=torch.float16,
    load_in_4bit=True,
)

model = FastLanguageModel.get_peft_model(
    model,
    r=16,
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj"],
    lora_alpha=16,
    lora_dropout=0,
    bias="none",
    use_gradient_checkpointing=True,
    random_state=42,
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
Unsloth: Failed to patch SmolVLMForConditionalGeneration forward function.
🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2025.4.1: Fast Llama patching. Transformers: 4.51.3.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 7.5. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.29.post3. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/762M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.37k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/438 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

Not an error, but Unsloth cannot patch MLP layers with our manual autograd engine since either LoRA adapters
are not enabled or a bias term (like in Qwen) is used.
Unsloth 2025.4.1 patched 22 layers with 22 QKV layers, 22 O layers and 0 MLP layers.


In [9]:
#3 Load and prepare OpenAssistant dataset
from datasets import load_dataset

dataset = load_dataset("Intel/orca_dpo_pairs", split="train[:10000]")

def format_orpo(example):
    return {
        "prompt": f"<|im_start|>user\n{example['question']}\n<|im_end|>",
        "chosen": f"<|im_start|>assistant\n{example['chosen']}\n<|im_end|>",
        "rejected": f"<|im_start|>assistant\n{example['rejected']}\n<|im_end|>",
    }

dataset = dataset.map(format_orpo)
print(f" Final usable examples: {len(dataset)}")

README.md:   0%|          | 0.00/196 [00:00<?, ?B/s]

orca_rlhf.jsonl:   0%|          | 0.00/36.3M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/12859 [00:00<?, ? examples/s]

Map:   0%|          | 0/10000 [00:00<?, ? examples/s]

 Final usable examples: 10000


In [11]:
#4 Train model using ORPOTrainer

from trl import ORPOTrainer, ORPOConfig

config = ORPOConfig(
    output_dir="tinyllama-orpo-intel",
    per_device_train_batch_size=2,
    gradient_accumulation_steps=2,
    learning_rate=5e-5,
    num_train_epochs=1,
    max_steps=100,
    logging_steps=5,
    save_strategy="no",
    report_to="none",
)

trainer = ORPOTrainer(
    model=model,
    tokenizer=tokenizer,
    train_dataset=dataset,
    args=config,
)

trainer.train()


/content/unsloth_compiled_cache/UnslothORPOTrainer.py:552: UserWarning: When using DPODataCollatorWithPadding, you should set `remove_unused_columns=False` in your TrainingArguments we have set it for you, but you should do it yourself in the future.
  warnings.warn(
==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 10,000 | Num Epochs = 1 | Total steps = 100
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 2
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 2 x 1) = 4
 "-____-"     Trainable parameters = 4,505,600/4,000,000,000 (0.11% trained)


Step,Training Loss,rewards / chosen,rewards / rejected,rewards / accuracies,rewards / margins,logps / rejected,logps / chosen,logits / rejected,logits / chosen,log_odds_ratio,log_odds_chosen,eval_logits / chosen,eval_logits / rejected,nll_loss
5,4.541600,-0.151917,-0.127412,0.200000,-0.024506,-1.274118,-1.519174,-2.868562,-2.348754,-0.921773,-0.338389,0,0,2.178645
10,4.243200,-0.133449,-0.132883,0.500000,-0.000565,-1.328834,-1.334485,-2.754775,-2.162274,-0.722439,-0.031018,No Log,No Log,2.049347
15,4.229700,-0.123110,-0.101236,0.100000,-0.021875,-1.012358,-1.231104,-2.749705,-2.042955,-0.881840,-0.318787,No Log,No Log,2.026656
20,4.250500,-0.132226,-0.110777,0.350000,-0.021449,-1.107770,-1.322261,-2.742855,-2.007662,-0.908862,-0.317451,No Log,No Log,2.034378
25,4.073100,-0.137863,-0.120095,0.300000,-0.017768,-1.200948,-1.378632,-2.730704,-2.231710,-0.851312,-0.250119,No Log,No Log,1.951403
30,3.766100,-0.122067,-0.106222,0.300000,-0.015845,-1.062222,-1.220667,-2.778332,-2.222068,-0.849646,-0.224033,No Log,No Log,1.798106
35,4.009700,-0.105807,-0.107686,0.500000,0.001879,-1.076860,-1.058066,-2.879885,-2.145356,-0.712566,0.028212,No Log,No Log,1.933595
40,3.993600,-0.111040,-0.109305,0.500000,-0.001735,-1.093053,-1.110400,-2.777297,-2.224131,-0.763974,0.014770,No Log,No Log,1.920423
45,3.923900,-0.108816,-0.101106,0.350000,-0.007710,-1.011056,-1.088161,-2.545233,-2.193883,-0.765895,-0.084685,No Log,No Log,1.885375
50,3.955000,-0.096360,-0.100671,0.450000,0.004311,-1.006710,-0.963595,-2.623728,-2.176910,-0.678008,0.135495,No Log,No Log,1.909693


TrainOutput(global_step=100, training_loss=3.8502572441101073, metrics={'train_runtime': 203.8676, 'train_samples_per_second': 1.962, 'train_steps_per_second': 0.491, 'total_flos': 0.0, 'train_loss': 3.8502572441101073, 'epoch': 0.04})

In [12]:
#5 Define generation function

def generate_response(prompt, max_tokens=128):
    chat_prompt = f"<|im_start|>user\n{prompt}\n<|im_end|>\n<|im_start|>assistant\n"
    inputs = tokenizer(chat_prompt, return_tensors="pt").to(model.device)
    outputs = model.generate(
        **inputs,
        max_new_tokens=max_tokens,
        temperature=0.7,
        top_p=0.9,
        pad_token_id=tokenizer.eos_token_id,
    )
    decoded = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return decoded.split("<|im_start|>assistant\n")[-1].split("<|im_end|>")[0].strip()

In [14]:
# Step 6: Test the fine-tuned ORPO model with clean prompts

def generate_response(prompt, max_tokens=128):
    chat_prompt = f"<|im_start|>user\n{prompt}\n<|im_end|>\n<|im_start|>assistant\n"
    inputs = tokenizer(chat_prompt, return_tensors="pt").to(model.device)
    outputs = model.generate(
        **inputs,
        max_new_tokens=max_tokens,
        temperature=0.7,
        top_p=0.9,
        pad_token_id=tokenizer.eos_token_id,
    )
    decoded = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return decoded.split("<|im_start|>assistant\n")[-1].split("<|im_end|>")[0].strip()

# Test different prompts
print("Test 1:")
print(generate_response("Can you explain quantum mechanics in basic terms?"))

print("\nTest 2:")
print(generate_response("List three benefits of exercising daily."))


print("\nTest 3:")
print(generate_response("How do airplanes stay in the air despite being so heavy?"))

print("\nTest 4:")
print(generate_response("What are some healthy snacks I can eat during work?"))

Test 1:
Sure! Quantum mechanics is a branch of physics that deals with the behavior of matter and energy at the smallest scales, known as the quantum level. It is a branch of physics that is based on the principles of quantum mechanics, which are the laws that govern the behavior of particles at the smallest scales.

Quantum mechanics is a fundamental theory that describes the behavior of particles at the smallest scales, known as the quantum level. It is based on the principles of quantum mechanics, which are the laws that govern the behavior of particles at the smallest scales.

Quantum mechanics is a branch of

Test 2:
1. Increased energy levels: Regular exercise helps to increase energy levels, which can help you to feel more alert and focused throughout the day.

2. Improved mood: Exercise has been shown to have a positive impact on mood, reducing symptoms of depression and anxiety.

3. Reduced stress: Regular exercise can help to reduce stress levels, which can have a positive im

In [15]:
# Step 7: Save the fine-tuned TinyLlama ORPO model

model.save_pretrained("tinyllama_orpo_intel")
tokenizer.save_pretrained("tinyllama_orpo_intel")

print("Fine-tuned ORPO model saved successfully.")

Fine-tuned ORPO model saved successfully.


In [16]:
# Step 8: Reload the saved fine-tuned TinyLlama ORPO model

from unsloth import FastLanguageModel

reloaded_model, reloaded_tokenizer = FastLanguageModel.from_pretrained(
    model_name="tinyllama_orpo_intel",
    max_seq_length=2048,
    dtype=torch.float16,
    load_in_4bit=True,
)

print("Model and tokenizer reloaded successfully.")

==((====))==  Unsloth 2025.4.1: Fast Llama patching. Transformers: 4.51.3.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 7.5. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.29.post3. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!
Model and tokenizer reloaded successfully.


In [17]:
# Step 9: Test the reloaded model on new prompts

def generate_response_reload(prompt, max_tokens=128):
    chat_prompt = f"<|im_start|>user\n{prompt}\n<|im_end|>\n<|im_start|>assistant\n"
    inputs = reloaded_tokenizer(chat_prompt, return_tensors="pt").to(reloaded_model.device)
    outputs = reloaded_model.generate(
        **inputs,
        max_new_tokens=max_tokens,
        temperature=0.7,
        top_p=0.9,
        pad_token_id=reloaded_tokenizer.eos_token_id,
    )
    decoded = reloaded_tokenizer.decode(outputs[0], skip_special_tokens=True)
    return decoded.split("<|im_start|>assistant\n")[-1].split("<|im_end|>")[0].strip()

# New unseen questions
print("New Test 1:")
print(generate_response_reload("How does gravity work in space?"))

print("\nNew Test 2:")
print(generate_response_reload("What are the main ingredients of a healthy salad?"))

print("\nNew Test 3:")
print(generate_response_reload("Explain why we have seasons on Earth."))

New Test 1:
Gravity in space is different from gravity on Earth. In space, gravity is caused by the curvature of space-time, which is caused by the mass of objects in space. The curvature of space-time is caused by the mass of objects in space, which is called the gravitational field.

In space, gravity is weaker than on Earth because there is no mass to distort space-time. Instead, the gravitational field is caused by the mass of objects in space. The mass of objects in space is called the gravitational field.

The gravitational field is caused by the mass of

New Test 2:
The main ingredients of a healthy salad are:
1. Leafy greens such as spinach, kale, or collard greens.
2. Fresh vegetables such as carrots, bell peppers, cucumbers, and tomatoes.
3. Nuts and seeds such as almonds, pumpkin seeds, or sunflower seeds.
4. Dairy products such as low-fat or non-fat milk, yogurt, or cheese.
5. Fat-free or low-fat

New Test 3:
The Earth has seasons because it is a planet that orbits around t

In [18]:
# Step 10: Analyze the output length for generated responses

prompts = [
    "Explain how the water cycle works.",
    "Give three tips to stay productive while working remotely.",
    "Describe the invention of the telephone in simple words."
]

for idx, prompt in enumerate(prompts, start=1):
    response = generate_response_reload(prompt)
    num_tokens = len(reloaded_tokenizer.tokenize(response))
    print(f"\nPrompt {idx} generated response with {num_tokens} tokens.")
    print(response)


Prompt 1 generated response with 126 tokens.
The water cycle is a complex process that involves the movement of water from one form to another. It is a fundamental process in the Earth's ecosystem, and it is essential for the survival of life on Earth.

The water cycle is divided into three main stages: evaporation, condensation, and precipitation. Evaporation is the process by which water vapor (usually in the form of water droplets) is converted into water molecules. This process occurs when the temperature of the Earth's surface rises, causing the water vapor to rise into the atmosphere.

Prompt 2 generated response with 129 tokens.
1. Set boundaries: Set specific times for work and avoid working outside of those hours.

2. Prioritize tasks: Prioritize tasks based on their importance and urgency.

3. Use productivity tools: Use productivity tools like time-tracking apps, task management apps, and productivity software to help you stay on track.

4. Stay connected: Stay connected wi